Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [9]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from pandas.io.json import json_normalize

BeautifulSoup Canada Wiki Data

In [10]:
# Get data from wikipedia to xml
wikipedia_canada ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_wikipedia_data = requests.get(wikipedia_canada).text
data_canada = BeautifulSoup(raw_wikipedia_data,'xml')

Create Dataframe Pandas

In [32]:
# Create table data canada

table = soup.find("table")
table_rows = table.tbody.find_all("tr")

res = []
for tr in table_rows:
    td = tr.find_all("td")
    row = [tr.text for tr in td]
    
    if row != [] and row[1] != "Not assigned":
        if "Not assigned" in row[2]: 
            row[2] = row[1]
        res.append(row)

# Dataframe with 3 columns
df = pd.DataFrame(res, columns = ["PostalCode", "Borough", "Neighborhood"])
df.head()

,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


Clean Data and Print Shape

In [42]:
# Data Cleanup

# Remove \n
df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df["PostalCode"] = df["PostalCode"].str.replace("\n","")

# Remove Duplicates
df = df.drop_duplicates()

# Remove Borough Not Assigned
df = df[df['Borough'] != 'Not assigned']

# Combine Nieghborhoods and sort
df = df.groupby(["PostalCode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()

#Print first 5 rows and shape
print("Shape: ", df.shape)
df.head()

Shape:  (103, 3)


,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [46]:
#Uplead and read CSV Geospatial_Coordinates derived from IBM Watson Studio
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_94ffe10e3def484b81e6caa1d0175c36 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Ont_pjzUgcZizt-pajMP7Lks83fnce_EKgSlnlxhIryH',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_94ffe10e3def484b81e6caa1d0175c36.get_object(Bucket='capstoneproject-donotdelete-pr-fxfx1jtoffaazi',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [57]:
# Merge Dataframes

df_data_1.columns=['PostalCode','Latitude','Longitude']
df_toronto = pd.merge(df, df_data_1, how = 'left', on = "PostalCode")
df_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
